<a href="https://colab.research.google.com/github/mc2259/Research_Response_Machine_Translation/blob/main/Research_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Importing Libraries

In [1]:
import pandas as pd
import json
import math
import os
from pathlib import Path
import random
import time
from tqdm.notebook import tqdm, trange
from typing import Dict, List, Set, Tuple

import numpy as np
import torch
import torch.nn as nn
from torch.nn import init
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, SubsetRandomSampler
from tqdm.notebook import tqdm, trange

# Processing the Dataset 
We make a pandas dataframe containing the columns chinese, reference, candidate, and the BLEU score.

In [59]:
  path='/content/drive/MyDrive/yoav/train.txt'
  
  with open(path, 'r', encoding='utf-8',errors='ignore') as f:
    file = f.read()
    
  path2='/content/drive/MyDrive/yoav/test.txt'
  
  with open(path2, 'r', encoding='utf-8',errors='ignore') as f2:
    file2 = f2.read()

In [60]:
datalist=file.split('\n\n')
testlist=file2.split('\n\n')
print(len(datalist))
print(len(testlist))

584
174


In [61]:
datadict={'chinese':[],'reference':[],'candidate':[],'score':[],'label':[]}
for example in datalist:
  example_list=example.split('\n')
  datadict['chinese'].append(example_list[0])
  datadict['reference'].append(example_list[1])
  datadict['candidate'].append(example_list[2])
  datadict['score'].append(example_list[3])
  datadict['label'].append(example_list[4])



In [62]:
testdict={'chinese':[],'reference':[],'candidate':[],'score':[],'label':[]}
for example in testlist:
  example_list=example.split('\n')
  testdict['chinese'].append(example_list[0])
  testdict['reference'].append(example_list[1])
  testdict['candidate'].append(example_list[2])
  testdict['score'].append(example_list[3])
  testdict['label'].append(example_list[4])

In [63]:
train_df=pd.DataFrame(datadict)
train_df.head()
test_df=pd.DataFrame(testdict)
test_df.head()

,chinese,reference,candidate,score,label
0,"发言人 说 , 世人 共 知 , 中国 的 发展 不仅 符合 13 亿 中国人民 的 根本利...",the spokesman noted that it is known to all th...,"the spokesman said , the world , a total of 13...",0.5100,M
1,"在 国际 政治 舞台上 , 中国 是 一个 负责任 的 国家 , 对 维护 世界和平 与 稳...","in the international political arena , china i...","in the international political arena , china i...",0.5996,H
2,"在 核 不 扩散 问题 上 , 中国 始终 严格 履行 有关 国际公约 , 坚决 反对 任何...","on the issue of nuclear non-proliferation , ch...",on the issue of non-proliferation of nuclear w...,0.6667,H
3,"发言人 指出 , 令人 遗憾 的 是 , 海德 先生 无视 客观 事实 , 在 中美 关系 ...",the spokesman points out that it is regrettabl...,the spokesman stated that it was regretful tha...,0.5326,H
4,中国 最 大规模 的 海水 淡化 工程 落户 舟山,china's largest seawater desalination project ...,china's largest seawater desalination project ...,0.6014,H


In [64]:
max_len=0
count=0
for i in train_df:
  max_len=max(max_len,len(train_df['reference'][count])+len(train_df['candidate'][count]))
  count+=1

In [65]:
max_len

485

The maximum total length of chinese+ reference + candidate is 614

# Feature Selection
We  focus on the syntactic and semantic structure of the candidate sentence and select features accordingly. Some features selected by us are:

1. The word embedding of the candidate sentence
2. The BLEU score
3. The word embedding of the reference sentence

We get the parse tree of a sentence and find out the sentence structure including density of constituent types to deisgn our features. We also use the t5 tokenizer to tokenise our sentences.






--- Word embedding of source sentence

--- Word embedding of reference sentence

--- Word embedding of candidate sentence

--- Parse tree of only candidate sentence

--- emotional agreement of only candidate sentence

--- lexical features of only the candidate sentence
    

In [8]:
!pip install transformers
!pip install datasets==1.0.2
!pip install sentencepiece


import datasets
import transformers
from transformers import T5Tokenizer, T5ForConditionalGeneration
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler

     |████████████████████████████████| 1.8MB 17.0MB/s 
     |████████████████████████████████| 890kB 42.4MB/s 
     |████████████████████████████████| 2.9MB 37.0MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=6eed92966ab48d88606a1db8c5fe5df8558327e5e49ae7e2ab764ca9cb3d91da
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
     |████████████████████████████████| 1.8MB 18.7MB/s 
     |████████████████████████████████| 245kB 52.1MB/s 
     |████████████████████████████████| 20.7MB 1.1MB/s 
  Found existing installation: pyarrow 0.14.1
    Uninstalling pyarrow-0.14.1:
      Successfully uninstalled pyarrow-0.14.1
     |████████████████████████████████| 1.2MB 17.3MB/s 


In [9]:
seed=42
torch.manual_seed(seed)
np.random.seed(seed)
torch.backends.cudnn.deterministic = True
source_len=150
target_len=512
tokenizer = T5Tokenizer.from_pretrained('t5-small')

In [10]:
class MT_Dataset(Dataset):

    def __init__(self, dataset):
        self.data = dataset
        self.source = self.data.chinese
        self.reference = self.data.reference
        self.candidate= self.data.candidate
        self.score=self.data.score
        self.label=self.data.label

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        # We clean the reference and the candidate
        source = str(self.source[index])
        source = ' '.join(source.split())
        reference = str(self.reference[index])
        reference = ' '.join(reference.split())
        candidate = str(self.candidate[index])
        candidate = ' '.join(candidate.split())
        return source,reference, candidate, float(self.score[index]),self.label[index]

In [67]:
train_dataset = MT_Dataset(train_df)
test_dataset = MT_Dataset(test_df)


In [66]:
len(train_df)

584

In [68]:
train_loader = DataLoader(train_dataset, batch_size=4)
test_loader = DataLoader(test_dataset, batch_size=4)


In [100]:
import spacy
nlp = spacy.load("en_core_web_sm")

In [101]:
doc=nlp('Maitryi is the best girl in the world')

In [146]:
vec=[a.pos_ for a in doc]
vec

['PROPN', 'AUX', 'DET', 'ADJ', 'NOUN', 'ADP', 'DET', 'NOUN']

# Model

In [14]:
class T5Translate(nn.Module):
  def __init__(self): 
    super(T5Translate, self).__init__()
    self.t5=T5ForConditionalGeneration.from_pretrained("t5-base")

  def forward(self,input_ids,attention_mask,labels):
    return self.t5.forward(input_ids,attention_mask,labels=labels)

  def compute_Loss(self, predicted_vector, gold_label):
    return self.loss(predicted_vector, gold_label)	
	
  def load_model(self, save_path):
    self.load_state_dict(torch.load(save_path))
	
  def save_model(self, save_path):
    torch.save(self.state_dict(), save_path)

In [15]:
model=T5Translate()

# Training

In [174]:
def train_epoch(model, train_loader,optimizer):
  model.train()
  total_loss = 0
  loss = 0
  batches=0
  for batch_source,batch_reference,batch_candidate,batch_score,batch_label in tqdm(train_loader, leave=False, desc="Training Batches"):
    optimizer.zero_grad()
    source=tokenizer.batch_encode_plus(batch_source, max_length=100,padding='max_length',return_tensors='pt',truncation=True)
    reference=tokenizer.batch_encode_plus(batch_reference, max_length=100,padding='max_length',return_tensors='pt',truncation=True)
    candidate=tokenizer.batch_encode_plus(batch_candidate, max_length=100,padding='max_length',return_tensors='pt',truncation=True)
    label=tokenizer.batch_encode_plus(batch_label, max_length=5,padding='max_length',return_tensors='pt',truncation=True)
    
    stack=[]
    for cand in batch_candidate:
      doc=nlp(cand)
      pos_vector=[a.pos_ for a in doc]
      noun_density=pos_vector.count('NOUN')/len(pos_vector)
      stack.append([noun_density])

    bleu=[]
    for score in batch_score:
      bleu.append([score])
    
    input_id=torch.cat((source['input_ids'],reference['input_ids'],candidate['input_ids'],torch.FloatTensor(stack),torch.FloatTensor(bleu)),1).to(device,dtype=torch.long)
    source_mask=torch.cat((source['attention_mask'],reference['attention_mask'],candidate['attention_mask'],torch.zeros(4,1),torch.zeros(4,1)),1).to(device,dtype=torch.long)
    target_ids =label['input_ids'].to(device,dtype=torch.long)

    
    outputs = model(input_ids = input_id, attention_mask = source_mask,labels = target_ids)
    generated_ids = model.t5.generate(
                input_ids = input_id,
                attention_mask = source_mask
                )
    preds = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=True) for g in generated_ids]
    # print(preds)
    loss=outputs[0]
    loss.backward()
    optimizer.step()
    total_loss += loss.item()
    # Saving checkpoints
    batches += 1
    if batches%500==0:
      model.save_model('/content/drive/MyDrive/yoav/t5_checkpoint')
  print('Loss on epoch: ', total_loss/batches)

In [160]:
lr= 1e-4
source_len=150
target_len=512
batch_size=2
num_epochs=2
seed=42
optimizer = torch.optim.Adam(params =  model.parameters(), lr=lr)
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
model=model.to(device)

In [175]:
for epoch in trange(3,desc='Epochs'):
        train_epoch(model, train_loader, optimizer)

Loss on epoch:  0.037129213052503704


Loss on epoch:  0.01928703393717376


Loss on epoch:  0.01467191274541744



# Inference

In [176]:
def compute_f1(a_gold, a_pred):
  """This function computs the f1 score betwwen 2 lists"""
  pred_tokens = a_pred
  truth_tokens = a_gold
  if len(pred_tokens) == 0 or len(truth_tokens) == 0:
    return int(pred_tokens == truth_tokens)

  count=0
  for i in range(len(pred_tokens)):
    if pred_tokens[i]==truth_tokens[i]:
      count+=1
    
    # if there are no common tokens then f1 = 0
  if count == 0:
    return 0
  
  prec = count / len(pred_tokens)
  rec = count / len(truth_tokens)

    
  return 2 * (prec * rec) / (prec + rec)

In [183]:
def evaluation(model, test_loader, optimizer):
  model.eval()
  predictions=[]
  gold_labels=[]
  with torch.no_grad():
    for batch_source,batch_reference,batch_candidate,batch_score,batch_label in tqdm(test_loader, leave=False, desc="Testing Batches"):
      source=tokenizer.batch_encode_plus(batch_source, max_length=100,padding='max_length',return_tensors='pt',truncation=True)
      reference=tokenizer.batch_encode_plus(batch_reference, max_length=100,padding='max_length',return_tensors='pt',truncation=True)
      candidate=tokenizer.batch_encode_plus(batch_candidate, max_length=100,padding='max_length',return_tensors='pt',truncation=True)
      label=tokenizer.batch_encode_plus(batch_label, max_length=5,padding='max_length',return_tensors='pt',truncation=True)
      input_id=torch.cat((source['input_ids'],reference['input_ids'],candidate['input_ids']),1).to(device,dtype=torch.long)
      source_mask=torch.cat((source['attention_mask'],reference['attention_mask'],candidate['attention_mask']),1).to(device,dtype=torch.long)

      target_ids =label['input_ids'].to(device,dtype=torch.long)
      generated_ids = model.t5.generate(
                input_ids = input_id,
                attention_mask = source_mask
                )

      preds = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=True) for g in generated_ids]
      predictions.extend(preds)
      gold_labels.extend(batch_label)
      
  return predictions, gold_labels

In [184]:
model=model.to(device)
predictions, actuals= evaluation(model, test_loader, optimizer)
final_df = pd.DataFrame({'Predictiona':predictions,'Gold labels':actuals})
final_df.to_csv('predictions.csv')

In [185]:
compute_f1(predictions,actuals)

0.8333333333333334

In [186]:
final_df

,Predictiona,Gold labels
0,M,M
1,H,H
2,M,H
3,H,H
4,H,H
...,...,...
169,M,M
170,H,M
171,H,M
172,H,M
